In [46]:
import rust_machine_learning
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from syed_ml_lib import *
import plotly.io as pio

In [47]:
# Get the csv data
df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
# df = pd.DataFrame(data={
#   "chest_pain": [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0],
#   "blocked_arteries": [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0],
#   "patient_weight": [205, 180, 210, 167, 156, 125, 168, 172],
#   "heart_disease": [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
# })
df  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [48]:
to_encode = [4]
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [55]:
# Prepare data for training with input rust functions expect
label_col = 1

train_data, test_data = split_train_test(encoded, 0.7, True)

# Remove any encoded data of the same class
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Get labels
train_data_labels = train_data.iloc[:, label_col].values.tolist()
train_data_features

,sepal length (cm),petal length (cm),petal width (cm),is_ class_Iris-setosa,is_ class_Iris-versicolor,is_ class_Iris-virginica
63,6.1,4.7,1.4,0.0,1.0,0.0
79,5.7,3.5,1.0,0.0,1.0,0.0
117,7.7,6.7,2.2,0.0,0.0,1.0
78,6.0,4.5,1.5,0.0,1.0,0.0
14,5.8,1.2,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...
134,6.1,5.6,1.4,0.0,0.0,1.0
50,7.0,4.7,1.4,0.0,1.0,0.0
143,6.8,5.9,2.3,0.0,0.0,1.0
106,4.9,4.5,1.7,0.0,0.0,1.0


In [50]:
# Now train the model
tree_depth = 5
forest_size = 100
learning_rate = 0.1
is_categorical = False
trained_model = rust_machine_learning.GradientBoost(
    train_data_features_list, train_data_labels, tree_depth, forest_size, learning_rate, is_categorical
)

In [51]:
# Now let us test the model
test_data_features = test_data.copy()
remove_encoded_category_data_and_label(test_data_features, feature_to_encoded_cols, label_col)
test_data_features = test_data_features.values.tolist()
test_data_labels = test_data[test_data.columns[label_col]].values.tolist()

labelled_results = trained_model.classify(test_data_features)

num_correct = 0
tolerance = 0.1
for test_label, result_label in zip(test_data_labels, labelled_results):
  num_correct += (abs(test_label-result_label)/(1.0 if test_label == 0 else test_label) <= tolerance)
print("Percent Correct: ", num_correct / len(test_data_labels))
labelled_results

Percent Correct:  0.6222222222222222


[2.9106381546716844,
 2.7816120817649557,
 2.710523501100848,
 3.5118575219697523,
 3.456073911798054,
 3.0713905663700776,
 2.784578606278967,
 3.622214020371958,
 2.600841848391728,
 2.4748499769041588,
 3.6632597398557842,
 3.24890690652703,
 2.7242827794964852,
 2.453269472853801,
 2.513914508947064,
 3.1638895725271357,
 2.6901326121489726,
 3.8164936775349427,
 2.427838182227952,
 3.6084736979426895,
 2.777270026062736,
 3.4262645095025785,
 2.999778463239022,
 3.1000752459866754,
 2.8737519670621547,
 3.2032163401894067,
 2.513914508947064,
 3.0454305506619237,
 2.930013638888188,
 3.218417431116577,
 3.1404162365179245,
 2.574930636614088,
 3.7764669592405133,
 3.3638643934669408,
 2.858237902119961,
 3.244855615553974,
 2.513914508947064,
 2.881814182491244,
 3.1002670737541895,
 2.1992323631718187,
 3.3193605761514933,
 3.2074399853376647,
 3.2551690565952653,
 3.759119097005115,
 3.659401693937679]